In [3]:
import pandas as pd
import numpy as np

In [13]:
soi = pd.DataFrame()
id_cols = ['COUNTYFIPS', 'COUNTYNAME', 'STATEFIPS']
for year in range(10,18):
    file_name = f'{year}incyallnoagi.csv'
    tmp = pd.read_csv(file_name, encoding='latin-1')
    list(tmp.columns)
    tmp['salt_amount_per_cap'] = (
            tmp['A18425'] + tmp['A18450'] + tmp['A18500']) / tmp['N1']
    tmp = tmp[id_cols + ['salt_amount_per_cap']]
    tmp['Year'] = int(f"20{year}")
    tmp = tmp.loc[tmp['COUNTYFIPS'] != 0]
    soi = pd.concat([soi, tmp], axis=0, ignore_index=True)
    soi = soi.sort_values(by=id_cols + ['Year'],
                      ascending=True)

soi.to_csv('all_years_soi.csv', index=False)

In [ ]:
soi = pd.read_csv('all_years_soi.csv')
soi_avg = soi.groupby(['COUNTYFIPS', 'STATEFIPS']).mean()[['salt_amount_per_cap']]
soi_avg = soi_avg.reset_index()
soi_avg['STATEFIPS']=soi_avg['STATEFIPS'].astype(str)
soi_avg['STATEFIPS']=soi_avg['STATEFIPS'].str.zfill(2)
soi_avg['STATEFIPS2']=soi_avg['STATEFIPS']
soi_avg['COUNTYFIPS']=soi_avg['COUNTYFIPS'].astype(str)
soi_avg['COUNTYFIPS']=soi_avg['COUNTYFIPS'].str.zfill(3)
soi_avg['area_fips']=soi_avg[['STATEFIPS2', 'COUNTYFIPS']].apply(lambda x: ''.join(x), axis=1)
soi_avg_pre = soi_avg[['area_fips','STATEFIPS', 'salt_amount_per_cap']]




In [ ]:
df1=pd.read_csv('master.csv')

In [4]:
df2=soi_avg_pre
df3=pd.merge(df1, df2, on="area_fips")

In [14]:
df3=df3.drop("Unnamed: 0", axis=1)

In [9]:
df3['treated'] = np.where(df3.salt_amount_per_cap > 10, 1, 0)

In [16]:
df3.to_csv('wage_salt.csv', index=False)

df3.head()

In [2]:
df3=pd.read_csv('wage_salt.csv')

In [3]:
df3.rename(columns = {"STATEFIPS": "state_fips"},
            inplace = True)

In [4]:
df3['treated'] = np.where(df3.salt_amount_per_cap > 10, 1, 0)


In [5]:
df3['post'] = np.where(df3.year > 2017, 1, 0)

In [5]:
df3['treated*post'] = df3.post * df3.treated

In [7]:
df3=df3.loc[df3['industry_code'] != 10]

In [9]:
df3.to_csv('wage_salt_v2.csv', index=False)

In [1]:
pip install statsmodels
pip install linearmodels

SyntaxError: invalid syntax (3270166653.py, line 1)

In [24]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels.panel import PooledOLS


In [5]:
df3=pd.read_csv('wage_salt_v2.csv')

In [10]:
df3.head()

industry_code               area_title           own_title  \
area_fips year                                                              
1001      2021            10  Autauga County, Alabama       Total Covered   
          2021            10  Autauga County, Alabama  Federal Government   
          2021            10  Autauga County, Alabama    State Government   
          2021            10  Autauga County, Alabama    Local Government   
          2021            10  Autauga County, Alabama             Private   

                          industry_title  annual_avg_emplvl  \
area_fips year                                                
1001      2021  10 Total, all industries              11123   
          2021  10 Total, all industries                 80   
          2021  10 Total, all industries                562   
          2021  10 Total, all industries               1874   
          2021  10 Total, all industries               8607   

                total_annual_wages  state_fips  salt_amount_per_cap  treated  \
area_fips year                                                                 
1001      2021           489569075           1             1.096738        0   
          2021             5000584           1             1.096738        0   
          2021            31105700           1             1.096738        0   
          2021            80715986           1             1.096738        0   
          2021           372746805           1             1.096738        0   

                post  treated*post  year state industry  
area_fips year                                           
1001      2021     1             0  2021     1       10  
          2021     1             0  2021     1       10  
          2021     1             0  2021     1       10  
          2021     1             0  2021     1       10  
          2021     1             0  2021     1       10

In [6]:
year = pd.Categorical(df3.year)
df3 = df3.set_index(['area_fips', 'year'])
df3['year'] = year


In [7]:
state = pd.Categorical(df3.state_fips)
df3['state'] = state

In [8]:
industry = pd.Categorical(df3.industry_code)
df3['industry'] = industry

In [16]:
df_wage_zero=df3.loc[df3['total_annual_wages'] == 0]
len(df_wage_zero.index) 

17463132

In [17]:
df3=df3.loc[df3['total_annual_wages'] != 0]

In [15]:
df_wage_zero.head()

industry_code             area_title           own_title  \
area_fips year                                                            
17095     2021         92611  Knox County, Illinois  Federal Government   
          2021        926110  Knox County, Illinois  Federal Government   

                                                   industry_title  \
area_fips year                                                      
17095     2021  NAICS 92611 Administration of general economic...   
          2021  NAICS 926110 Administration of general economi...   

                annual_avg_emplvl  total_annual_wages  state_fips  \
area_fips year                                                      
17095     2021                  0                  10          17   
          2021                  0                  10          17   

                salt_amount_per_cap  treated  post  treated*post  year state  \
area_fips year                                                                 
17095     2021             1.325571        0     1             0  2021    17   
          2021             1.325571        0     1             0  2021    17   

               industry  
area_fips year           
17095     2021    92611  
          2021   926110

In [14]:
df5=df3.iloc[100000:2000000]

In [18]:
df5.tail()

,industry_code,area_title,own_title,industry_title,annual_avg_emplvl,total_annual_wages,state_fips,salt_amount_per_cap,treated,post,treated*post,state,industry,year
1999995,923120,"San Diego County, California",State Government,NAICS 923120 Administration of public health p...,0,0,6,4.963774,0,1,0,6,923120,2018
1999996,92313,"San Diego County, California",Federal Government,NAICS 92313 Other human resource programs admi...,372,33977974,6,4.963774,0,1,0,6,92313,2018
1999997,92313,"San Diego County, California",State Government,NAICS 92313 Other human resource programs admi...,983,76855234,6,4.963774,0,1,0,6,92313,2018
1999998,92313,"San Diego County, California",Local Government,NAICS 92313 Other human resource programs admi...,0,0,6,4.963774,0,1,0,6,92313,2018
1999999,923130,"San Diego County, California",Federal Government,NAICS 923130 Other human resource programs adm...,372,33977974,6,4.963774,0,1,0,6,923130,2018


In [26]:
df3.dtypes

industry_code            object
area_title               object
own_title                object
industry_title           object
annual_avg_emplvl         int64
total_annual_wages        int64
state_fips                int64
salt_amount_per_cap     float64
treated                   int64
post                      int64
treated*post              int64
year                   category
state                  category
industry               category
dtype: object

In [28]:
logwage = np.log(df3.total_annual_wages)

In [29]:
exog_vars = ['treated*post', 'year']
exog = sm.add_constant(df3[exog_vars])
mod = PooledOLS(logwage, exog)
pooled_res = mod.fit()
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:     total_annual_wages   R-squared:                        0.0066
Estimator:                  PooledOLS   R-squared (Between):             -0.7435
No. Observations:            10158863   R-squared (Within):            2.526e-05
Date:                Sat, Nov 19 2022   R-squared (Overall):              0.0066
Time:                        16:59:51   Log-likelihood                -2.164e+07
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      6721.6
Entities:                        3144   P-value                           0.0000
Avg Obs:                       3231.2   Distribution:             F(10,10158852)
Min Obs:                       84.000                                           
Max Obs:                    2.203e+04   F-statistic (robust):             6721.6
                            

In [23]:
dir(smf)

NameError: name 'smf' is not defined

In [27]:
FE_ols = smf.ols(formula='logwage ~ 1 + treated*post + C(year)',
data = df3).fit()
print(FE_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                logwage   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                 1.184e+04
Date:                Sat, 19 Nov 2022   Prob (F-statistic):               0.00
Time:                        16:52:25   Log-Likelihood:            -2.1608e+07
No. Observations:            10158863   AIC:                         4.322e+07
Df Residuals:                10158851   BIC:                         4.322e+07
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          15.0007      0.002   72

In [ ]:
FE_ols = smf.ols(formula='logwage ~ 1 + treated*post + C(year) + C(state)',
data = df3).fit()
print(FE_ols.summary())